物語IDを選択

### 対象物語 (随時追加予定)
| num | gutenbergID | name | #words | condition |
| ---- | ---- | ---- | ---- | ---- |
| 1 | 18155 | The Story of the Three Little Pigs | 960 | fin |
| 2 | 1661 | The Adventures of Sherlock Holmes\* | 8,520 | fin |
| 3 | 11 |  Alice's Adventures in Wonderland by Lewis Carroll | 26,520 | fin |
| 4 | 120 | TREASURE ISLAND | 67,870 | fin |
| 5 | 829 | GULLIVER'S TRAVELS | 105,080 | fin |
| 6 | 12 | Through the Looking-Glass by Lewis Carroll | 29,256 | fin |
| 7 | 35 | The Time Machine by H. G. Wells\** | 32,376 | fin |
| 8 | 16 | Peter Pan by J. M. Barrie | 47,112 | fin |
| 9 | 146 | A Little Princess by Frances Hodgson Burnett | 66,264 | fin |
| 10 | 164 | Twenty Thousand Leagues under the Sea by Jules Verne\** | 104,335 | fin |
| 11 | 521 | ~~The Life and Adventures of Robinson Crusoe by Daniel Defoe~~\*** | 120,792 | fin |
| 12 | 1257 | The three musketeers by Alexandre Dumas and Auguste Maquet | 228,997 | fin |



In [1]:
#インポート
import glob
from tqdm import tqdm
import itertools
import os
import json
import difflib
import Levenshtein
import textwrap
from difflib import SequenceMatcher

In [2]:
print(SequenceMatcher(None, 'Dinah (cat)', 'Dinah').ratio())
print(SequenceMatcher(None, 'Dinah (cat)', 'Alice').ratio())

0.625
0.25


In [3]:
#IDと物語名の辞書を構築しておく
story_d={
    "The Story of the Three Little Pigs":'18155',
    "The Adventures of Sherlock Holmes":'1661',
    "Alice's Adventures in Wonderland":'11',
    "Treasure Island":'120',
    "Gulliver's Travels":'829',
    "Through the Looking-Glass":"12",
    "The Time Machine":"35",
    "Peter Pan":"16",
    "A Little Princess":"146",
    "Twenty Thousand Leagues under the Sea":"164",
    "The Three Musketeers":"1257",
}

In [4]:
#可変の変数
size_magnification = 30 #サイズの倍率(変数)10 重要度とかける
image_min_size=15 #画像の最小サイズ
max_edge_label_len=40 #edgeのlabelの文字数を最大何文字にするか

#sample0について実行
sample_file='sample0'

テキストファイルから情報を抽出  
全ての物語でこのプログラムを動かしたら抽出できるようにする

# 処理を関数化する

## 章，節名の抽出

### sectionがある場合

In [5]:
#物語ことに処理を追加（手動）

def create_section(ID,divide_count,chapter_list,chapter_name_list,section_name_list):
  chapter_num=0 #章番号
  section_num=0 #節番号

  for i in range(divide_count):
    chapter_file = open(f'{ID}/body_scene{i}.txt', 'r', encoding="utf-8")
    lines = chapter_file.readlines()

    #section_nameの決定(IDごとに異なる)
    if ID=='120':
      #章になっているとき
      if i in chapter_list:
        chapter_name=lines[0].strip() #ファイルの最初の行をchapter_titleとする
        chapter_name_list.append(chapter_name)

        #章を更新すると，節の番号はリセット
        chapter_num+=1
        section_num=1

        #章があるときは三行目に節タイトル
        section_name=f'chapter{chapter_num}_{section_num} {i+1} {lines[2].strip()}'

      else:
        #章タイトルがないとき
        #節のタイトルは2行目
        section_num+=1
        section_name=f'chapter{chapter_num}_{section_num} {i+1} {lines[1].strip()}'

    elif ID=='1661':
      #章になっているとき
      if i in chapter_list:
        chapter_name=lines[0].strip() #ファイルの最初の行をchapter_titleとする
        chapter_name_list.append(chapter_name)
        #章を更新すると，節の番号はリセット
        chapter_num+=1
        section_num=1
      #節のタイトルは適当に付ける
      section_name=f'section{chapter_num}_{section_num}'
      section_num+=1

    elif ID=='829':
      if i in chapter_list:
        chapter_name=lines[0].strip() #ファイルの最初の行をchapter_titleとする
        chapter_name_list.append(chapter_name)
        #章を更新すると，節の番号はリセット
        chapter_num+=1
        section_num=1
        #0番目と1番目が少し特殊なので場合分け
        if i==0 or i==1:
          section_name=f'chapter{chapter_num}_{section_num}'
        else:
          #章があるときは2行目に節タイトル
          section_name=f'chapter{chapter_num}_{section_num} {lines[1].strip()}'
      else:
        # 章タイトルがないとき
        #節のタイトルは1行目
        section_name=f'chapter{chapter_num}_{section_num} {lines[0].strip()}'
        section_num+=1
        
    
    elif ID=='164':
      if i in chapter_list:
        chapter_name=lines[0].strip() #ファイルの最初の行をchapter_titleとする
        chapter_name_list.append(chapter_name)
        #章があるときは2行目に節タイトル
        section_name=lines[1].strip()
      else:
        #章がないときは1行目に節タイトル
        section_name=lines[0].strip()

    #節名を追加
    section_name_list.append(section_name)

### sectionがない場合

In [6]:
def create_non_section(ID,divide_count,chapter_name_list):
  if ID=='18155':
    chapter_name_list.append('Chapter1')
  elif (ID=='35') or (ID=='146'):
    for i in range(divide_count):
      #chapter_nameの決定
      chapter_file = open(f'{ID}/body_scene{i}.txt', 'r', encoding="utf-8")
      lines = chapter_file.readlines()
      chapter_name=f"{lines[0].strip()} {lines[1].strip()}"#ファイルの最初の行と2行目を合わせたものをchapter_titleとする
      chapter_name_list.append(chapter_name)
  else: #11,12,16,1257
    for i in range(divide_count):
      #chapter_nameの決定
      chapter_file = open(f'{ID}/body_scene{i}.txt', 'r', encoding="utf-8")
      lines = chapter_file.readlines()
      chapter_name=lines[0].strip() #ファイルの最初の行をchapter_titleとする
      chapter_name_list.append(chapter_name)

## nodeとedgeの処理

In [7]:
def create_node_edge(ID,divide_count,node_list,edge_list,sample_file='sample0',size_magnification =10,image_min_size=20):
  node_id_num=0
  edge_id_num=0

  for i in range(divide_count):
    node_list_i=[]
    edge_list_i=[]

    #nodeについての処理
    node_file = open(f'./{ID}/{sample_file}/node_scene{i}.txt', 'r', encoding="utf-8")
    lines = node_file.readlines()    
    
    for j in range(len(lines)):
      line=lines[j]
      #左から分割
      lists = line.split(',',1)

      #右から分割
      r_lists=lists[1].rsplit(',',1)

      #抽出
      label = lists[0].strip()
      title=r_lists[0].strip()
      
      #nodeのtitleが長すぎる箇所は改行
      #edge_titleが長すぎる場合に改行
      node_title_list=textwrap.wrap(title,100)
      title='<br>'.join(node_title_list)


      #重要度がしっかりしていない箇所がある（その場合重要度0）
      try:
        important_rate=float(r_lists[1].strip())

      except ValueError:
        important_rate=0

      #サイズはimportant_rateを基に決定
      size=int(image_min_size+important_rate*size_magnification)

      #辞書化する(idは適当に番号文字列)
      node_d={
          "id":f'node_id{node_id_num}',
          "label":label,
          "title":title,
          "size":size,
        }

      node_list_i.append(node_d)

      node_id_num+=1


    #nodeの統合処理をするならここで処理

    #全体のリストに追加
    node_list.append(node_list_i)


    #edgeについての処理
    edge_file = open(f'./{ID}/{sample_file}/edge_scene{i}.txt', 'r', encoding="utf-8")
    lines = edge_file.readlines()
        

    for j in range(len(lines)):
      line=lines[j]
      #左から分割
      lists = line.split(',',1)

      lists2=lists[1].split(',',1)

      lists3=lists2[1].split(',',1)


      #抽出
      edge_from = lists[0].strip()
      edge_to = lists2[0].strip()
      edge_label=lists3[0].strip()
      edge_title=lists3[1].strip()
      
      #以下edge_labelの例外処理
      #edge_labelに / が含まれる場合その前だけ出力
      if '/' in edge_label:
        edge_label=edge_label.split('/')[0]
      #edge_labelに & が含まれる場合その前だけ出力
      if '&' in edge_label:
        edge_label=edge_label.split('&')[0]
      #edge_labelが長すぎる場合に省略(字数制限)
      edge_label=textwrap.shorten(edge_label, max_edge_label_len, placeholder='~')
        
      #edge_titleが長すぎる場合に改行
      edge_title_list=textwrap.wrap(edge_title,100)
      edge_title='<br>'.join(edge_title_list)


      #from と to　を示すものが nodeにあるか確認
      #あった場合そのidを抽出し，edge_from_idとedge_to_idに与える
      from_flag=0
      to_flag=0

      for k in range(len(node_list_i)):
        #node_list_i[k]は辞書を表す
        if (from_flag==0) and (node_list_i[k]['label']==edge_from):
          edge_from_id= node_list_i[k]['id']
          from_flag=1


        if (to_flag==0) and (node_list_i[k]['label']==edge_to):
          edge_to_id= node_list_i[k]['id']
          to_flag=1
        
        if from_flag ==1 and to_flag==1:
            break

      #どちらかなかったら近いnodeを類似度により探す
      if (from_flag==0):
        max_similarity = 0
        for k in range(len(node_list_i)):  
            dist = SequenceMatcher(None,edge_from,node_list_i[k]['label']).ratio()
            if max_similarity < dist:
                max_similarity=dist
                edge_from_id = node_list_i[k]['id']
        
      if (to_flag==0):
         max_similarity = 0
         for k in range(len(node_list_i)):  
             dist = SequenceMatcher(None,edge_to,node_list_i[k]['label']).ratio()
             if max_similarity <= dist:
                 max_similarity=dist
                 edge_to_id = node_list_i[k]['id']
        
            
        
        
      #過去に出てきた例とedge_from_idとedge_to_idが完全一致してしまった場合
      #edge_labelは最初の1つだけ使用，edge_titleは<br>でくっつける
      match_flag=0
      for k in range(len(edge_list_i)):
        #edge_list_i[k]は辞書を表す
        if (edge_list_i[k]['from']==edge_from_id) and (edge_list_i[k]['to']==edge_to_id):
            if edge_list_i[k]['title'][0] !='･':
                 edge_list_i[k]['title']=f"･ {edge_list_i[k]['title']}"
            edge_list_i[k]['title']= f"{edge_list_i[k]['title']}<br>･ {edge_title}"
            match_flag=1
            break

      #過去に出てきた例とedge_from_idとedge_to_idが完全一致しなかった場合
      if match_flag==0:
        #辞書化する(idは一意の番号文字列,矢印を追加(allows))
          edge_d={
              "id":f'edge_id{edge_id_num}',
              "label":edge_label,
              "title":edge_title,
              "from":edge_from_id,
              "to":edge_to_id,
              "arrows": "to",
            }

          #辞書を追加
          edge_list_i.append(edge_d)

          #edgeのnumを増やす
          edge_id_num+=1


    #edgeの統合処理をするならここで処理
    #edge_list_i内でfrom to と to fromが同じものが存在する場合，矢印をカーブさせる
    for pair in itertools.combinations(edge_list_i, 2):
      if (pair[0]['from']==pair[1]['to']) and (pair[1]['from']==pair[0]['to']):
        f_t=pair[0]['id']
        t_f=pair[1]['id']
        for j in range(len(edge_list_i)):
          if edge_list_i[j]['id']== f_t or t_f:
            edge_list_i[j]['smooth']= {
                                      "type": "curvedCW",
                                      "roundness": 0.3
                                  }

    #全体のリストに追加
    edge_list.append(edge_list_i)

## summaryの処理

In [8]:
 def create_summary(ID,divide_count,summary_list,sample_file='sample0'):
  for i in range(divide_count):
    summary_file = open(f'{ID}/{sample_file}/summary.txt', 'r', encoding="utf-8")
    lines = summary_file.readlines()
    summary=lines[i].strip()
    summary_list.append(summary)

## 外部ファイルに出力


### sectionがある場合

In [9]:
def section_json(chapter_list,summarized_data):
  for i in range(len(chapter_list)):
    chapter_summarized_data=[]

    #chapterがあるbody_sceneについてその間の節を抜き出していく
    if i!=len(chapter_list)-1:
      for j in range(chapter_list[i],chapter_list[i+1]):

        summarized_file={
            "sectionName":section_name_list[j],
            "summary":summary_list[j],
            "nodes":node_list[j],
            "edges":edge_list[j],
        }

        chapter_summarized_data.append(summarized_file)
    #chapterがあるbody_sceneについて，最後のchapterからは最後まで
    else:
      for j in range(chapter_list[i],divide_count):

        summarized_file={
            "sectionName":section_name_list[j],
            "summary":summary_list[j],
            "nodes":node_list[j],
            "edges":edge_list[j],
        }

        chapter_summarized_data.append(summarized_file)


    summarized_folder={
        "sectionName":chapter_name_list[i],
        "subSection":chapter_summarized_data,
    }

    summarized_data.append(summarized_folder)

### sectionがない場合

In [10]:
def non_section_json(divide_count,summarized_data):
  for i in range(divide_count):
    summarized_file={
        "sectionName":chapter_name_list[i],
        "summary":summary_list[i],
        "nodes":node_list[i],
        "edges":edge_list[i],
    }

    summarized_data.append(summarized_file)

# 全体的な処理

In [11]:
for n in range (len(story_d)):
    os.chdir('log')
    title_name=list(story_d.keys())[n]
    ID=story_d[title_name]
    print(f'ID : {ID} title : {title_name}')
    
    
    #事前に変数定義
    node_list=[]
    edge_list=[]
    summary_list=[]
    chapter_name_list=[]
    section_name_list=[]
    summarized_data=[]

    section_json_flag=-1


    #分割数をカウント（divide_count）
    divide_count=len(glob.glob(f'./{ID}/body_scene*.txt', recursive=False))
    print(divide_count)
    
    #chapter_listの作成（章と節がともに存在する場合のみ）
    #章が何番で区切られているか確認してリストを作成(話によって異なる)
    if ID=='120':
      chapter_list=[0,6,12,15,21,27]
      section_json_flag=1

    elif ID=='1661':
      chapter_list=[0]
      section_json_flag=1

    elif ID=='829':
      #partが一番大きなくくり，次点がchapterなので，part→chapter,chapter→sectionに対応させる
      chapter_list=[0,1,2,10,18,29]
      section_json_flag=1

    elif ID=='164':
      #partが一番大きなくくり，次点がchapterなので，part→chapter,chapter→sectionに対応させる
      chapter_list=[0,23]
      section_json_flag=1

    else:
       chapter_list=[0]


    if section_json_flag==1:
      create_section(ID,divide_count,chapter_list,chapter_name_list,section_name_list)
    else:
      create_non_section(ID,divide_count,chapter_name_list)

    create_node_edge(ID,divide_count,node_list,edge_list,sample_file,size_magnification,image_min_size)

    create_summary(ID,divide_count,summary_list,sample_file)

    if section_json_flag==1:
      section_json(chapter_list,summarized_data)
    else:
      non_section_json(divide_count,summarized_data)
    
    new_dir_path_recursive=f'../summarized_data/{ID}/{sample_file}'
    os.makedirs(new_dir_path_recursive, exist_ok=True)
    os.chdir(new_dir_path_recursive)
    
    with open("all_data.json", "w", encoding="utf-8") as f:
        json.dump(summarized_data,f,indent=4)
    f.close()
    
    with open("../title.txt", "w", encoding="utf-8") as f:
        f.write(title_name)
    f.close()
    
    os.chdir('../../../')
    
    print(f'node_list\n{node_list}')
    print(f'\nedge_list\n{edge_list}')
    print(f'\nsummary_list\n{summary_list}')
    print(f'\nchapter_name_list\n{chapter_name_list}')
    print(f'\nsection_name_list\n{section_name_list}')
    print(f'\nsummarized_data\n{summarized_data}')

ID : 18155 title : The Story of the Three Little Pigs
1
node_list
[[{'id': 'node_id0', 'label': 'Old Sow', 'title': 'The mother of the three little pigs, she sent them out to seek their fortune due to lack of<br>resources', 'size': 19}, {'id': 'node_id1', 'label': 'First Little Pig', 'title': 'The pig that built a house of straw and was eaten by the Wolf', 'size': 31}, {'id': 'node_id2', 'label': 'Man with straw', 'title': 'Gave the first little pig straw to build a house', 'size': 18}, {'id': 'node_id3', 'label': 'Wolf', 'title': 'The main antagonist who blew down the houses of straw and furze and ate the first two pigs, failed<br>to destroy the brick house, and was ultimately outsmarted and cooked by the third pig', 'size': 43}, {'id': 'node_id4', 'label': 'Second Little Pig', 'title': 'The pig that built a house of furze and was eaten by the Wolf', 'size': 30}, {'id': 'node_id5', 'label': 'Man with furze', 'title': 'Gave the second little pig furze to build a house', 'size': 18}, {'

node_list
[[{'id': 'node_id0', 'label': 'Alice', 'title': 'The main character of the story who falls down a rabbit-hole into a fantastical world', 'size': 45}, {'id': 'node_id1', 'label': "Alice's sister", 'title': "Alice's older sister who reads a book without pictures or conversations", 'size': 27}, {'id': 'node_id2', 'label': 'White Rabbit', 'title': 'A talking rabbit with pink eyes and a waistcoat who Alice follows down the rabbit-hole', 'size': 39}, {'id': 'node_id3', 'label': 'Dinah', 'title': "Alice's cat who she thinks about during her fall down the rabbit-hole", 'size': 24}], [{'id': 'node_id4', 'label': 'Alice', 'title': 'The main character, who experiences size changes and adventures in a fantastical world', 'size': 45}, {'id': 'node_id5', 'label': 'White Rabbit', 'title': 'A character who Alice follows into Wonderland, prompting her adventures', 'size': 39}, {'id': 'node_id6', 'label': 'Duchess', 'title': 'Mentioned by the White Rabbit, a character that Alice thinks about e

node_list
[[{'id': 'node_id0', 'label': 'Jim Hawkins', 'title': 'The narrator of the story, who recounts his adventure about Treasure Island', 'size': 45}, {'id': 'node_id1', 'label': 'Squire Trelawney', 'title': 'A local gentleman who finances the voyage to Treasure Island', 'size': 36}, {'id': 'node_id2', 'label': 'Dr. Livesey', 'title': 'A local doctor and magistrate; a level-headed leader of the expedition to Treasure Island', 'size': 39}, {'id': 'node_id3', 'label': 'Captain', 'title': 'The anonymous old seaman who takes up lodging at the Admiral Benbow inn and has a mysterious past<br>involving pirates', 'size': 42}, {'id': 'node_id4', 'label': "Jim's father", 'title': 'The keeper of the Admiral Benbow inn where the old seaman takes up lodging', 'size': 27}, {'id': 'node_id5', 'label': 'Man with the barrow', 'title': "The person who brings the captain's sea-chest to the Admiral Benbow inn", 'size': 21}, {'id': 'node_id6', 'label': 'Seafaring man with one leg', 'title': 'A person 

node_list
[[{'id': 'node_id0', 'label': 'Mr. Lemuel Gulliver', 'title': 'The author of the Travels and a person of reputed veracity among his neighbours', 'size': 42}, {'id': 'node_id1', 'label': 'The Publisher', 'title': "The individual writing the preface and responsible for bringing Mr. Gulliver's Travels to the public", 'size': 36}, {'id': 'node_id2', 'label': 'Richard Sympson', 'title': 'The person addressing the reader in the preface and claiming a familial relation to Mr. Gulliver', 'size': 39}], [{'id': 'node_id3', 'label': 'Captain Gulliver', 'title': 'The protagonist and narrator of the letter, expressing his disappointment in the publication and<br>reception of his accounts of his travels', 'size': 45}, {'id': 'node_id4', 'label': 'Cousin Sympson', 'title': 'The recipient of the letter, urged Gulliver to publish his travels and may have been responsible for<br>alterations', 'size': 39}, {'id': 'node_id5', 'label': 'Queen Anne', 'title': 'A monarch mentioned to have been fals

node_list
[[{'id': 'node_id0', 'label': 'White Kitten', 'title': 'One of the two kittens belonging to Dinah; it had been having its face washed by the old cat and was<br>not involved in the earlier mischief caused by the Black Kitten', 'size': 33}, {'id': 'node_id1', 'label': 'Black Kitten', 'title': "The mischievous kitten of Dinah that caused the mischief with Alice's ball of worsted", 'size': 36}, {'id': 'node_id2', 'label': 'Dinah', 'title': 'The old cat who is the mother of the White and Black Kittens and was washing their faces', 'size': 39}, {'id': 'node_id3', 'label': 'Alice', 'title': 'The protagonist who interacts with the kittens and imagines a series of fantastical scenarios<br>involving them and other characters', 'size': 45}, {'id': 'node_id4', 'label': 'Red Queen', 'title': 'A character from "Through the Looking-Glass" used by Alice to pretend and imagine; Alice tries to<br>make the Black Kitten impersonate it', 'size': 33}, {'id': 'node_id5', 'label': 'White Queen', 'ti

node_list
[[{'id': 'node_id0', 'label': 'The Time Traveller', 'title': 'The main character who is discussing complex ideas about space and time with his guests', 'size': 45}, {'id': 'node_id1', 'label': 'Filby', 'title': "An argumentative person with red hair, part of the Time Traveller's audience and a skeptic of his<br>theories", 'size': 33}, {'id': 'node_id2', 'label': 'Psychologist', 'title': "A character who engages in the discussion about the Time Traveller's theories", 'size': 33}, {'id': 'node_id3', 'label': 'The Medical Man', 'title': 'Another guest who participates in the conversation, particularly in relation to the movement in<br>space and time', 'size': 33}, {'id': 'node_id4', 'label': 'The Very Young Man', 'title': 'A character who attempts to relight his cigar and participates in the discussion with simpler input', 'size': 30}, {'id': 'node_id5', 'label': 'The Provincial Mayor', 'title': "A character who is trying to understand the Time Traveller's theories", 'size': 30}

node_list
[[{'id': 'node_id0', 'label': 'Wendy', 'title': "The eldest of the Darling children, focus of her parents' attention and the first to encounter Peter<br>Pan", 'size': 39}, {'id': 'node_id1', 'label': 'Mrs. Darling', 'title': "The mother of the Darling children, romantic and caring figure, concerned about her children's<br>future and safety", 'size': 37}, {'id': 'node_id2', 'label': 'Mr. Darling', 'title': 'The father of the Darling children, practical and proud but sometimes insecure about how he is<br>perceived by others', 'size': 34}, {'id': 'node_id3', 'label': 'John', 'title': "The second child of the Darling family, part of the children's adventures and experiences", 'size': 30}, {'id': 'node_id4', 'label': 'Michael', 'title': 'The youngest of the Darling children, depicted as innocent and carefree, also part of the adventures', 'size': 30}, {'id': 'node_id5', 'label': 'Nana', 'title': "The Darling family's nursemaid dog, protector and caretaker of the children", 'size':

node_list
[[{'id': 'node_id0', 'label': 'Sara Crewe', 'title': 'The central character who is a thoughtful and dreamy seven-year-old girl', 'size': 43}, {'id': 'node_id1', 'label': 'Captain Crewe', 'title': "Sara's affectionate and wealthy father who has to leave her at a boarding school", 'size': 40}, {'id': 'node_id2', 'label': 'Miss Minchin', 'title': 'The strict and cold headmistress of the boarding school Sara is left at', 'size': 36}, {'id': 'node_id3', 'label': 'Emily', 'title': "The doll that Sara and her father purchase, which becomes Sara's companion", 'size': 33}, {'id': 'node_id4', 'label': 'Miss Amelia', 'title': "Miss Minchin's more good-natured but submissive sister", 'size': 27}, {'id': 'node_id5', 'label': 'Lady Meredith', 'title': 'A character mentioned as having recommended the boarding school to Captain Crewe', 'size': 21}], [{'id': 'node_id6', 'label': 'Sara Crewe', 'title': "The protagonist, a new pupil at Miss Minchin's school, fluent in French due to her upbringi

node_list
[[{'id': 'node_id0', 'label': 'Cetacean', 'title': 'A mysterious and puzzling sea creature causing excitement and concern', 'size': 43}, {'id': 'node_id1', 'label': 'Captain Baker', 'title': 'Captain of the steamer Governor Higginson who encounters the cetacean', 'size': 31}, {'id': 'node_id2', 'label': 'Captain Anderson', 'title': "Captain of the Scotia who dealt with the aftermath of the creature's attack", 'size': 33}, {'id': 'node_id3', 'label': 'Merchants', 'title': "Individuals excited and concerned about the sea creature's appearance affecting maritime activities", 'size': 28}, {'id': 'node_id4', 'label': 'Common Sailors', 'title': 'People at sea who are excited and concerned about the encounters with the mysterious creature', 'size': 28}, {'id': 'node_id5', 'label': 'Captains of Vessels', 'title': 'Individuals in charge of ships that have been disturbed by the cetacean', 'size': 30}, {'id': 'node_id6', 'label': 'Skippers', 'title': "Ship masters affected by the sea cr

node_list
[[{'id': 'node_id0', 'label': 'M. d’Artagnan', 'title': 'The person whose memoirs the narrator stumbled upon; inspired the story', 'size': 42}, {'id': 'node_id1', 'label': 'Louis XIV', 'title': "The king during the period the author made researches; the subject of the author's history research", 'size': 33}, {'id': 'node_id2', 'label': 'Pierre Rouge', 'title': 'The printer of M. d’Artagnan’s memoirs; symbolizes the prohibited truth-telling of the era', 'size': 27}, {'id': 'node_id3', 'label': 'Louis XIII', 'title': "Mentioned as part of the era's portraits in d’Artagnan’s memoirs", 'size': 30}, {'id': 'node_id4', 'label': 'Anne of Austria', 'title': "Mentioned as part of the era's portraits in d’Artagnan’s memoirs", 'size': 30}, {'id': 'node_id5', 'label': 'Richelieu', 'title': "Mentioned as part of the era's portraits in d’Artagnan’s memoirs", 'size': 30}, {'id': 'node_id6', 'label': 'Mazarin', 'title': "Mentioned as part of the era's portraits in d’Artagnan’s memoirs", 'siz